In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import seaborn as sns
import requests

In [14]:
cid = "6540d0625e24409ea4f83ff44c9ed3b3"
secret = "81c6110978424fe3be0e23edda134f70"
genius_key = "6mOCQw6G-wXsiEqXgiWa54JYyZXqNo3B9oOFBVli96oO2girdJVYcnzmesC_p5vy"

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
# playlist_link = "https://open.spotify.com/playlist/5MGi3muFW4A7exL52WaFua?si=c1f91ee9ee044266" # MY DG PLAYLIST
playlist_link = "https://open.spotify.com/playlist/1HX3Xw3Uo9P7UHtCUh27f0?si=f3ff6e8e171f4b6c" # ALL DG Songs
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [15]:
class GetLyrics():
    
    def __init__(self, spotify_client_id, spotify_client_secret, playlist_id, genius_key):
        self.spotify_client_id = spotify_client_id
        self.spotify_client_secret = spotify_client_secret
        # self.user_id = user_id
        self.playlist_id = playlist_id
        self.genius_key = genius_key
        
    def get_playlist_info(self):
        token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
        sp = spotipy.Spotify(token)
        playlist = sp.playlist_tracks(self.playlist_id)
        self.playlist = playlist
        return self.playlist
    
    def get_track_names(self):
        track_names = []
        for song in range(len(self.playlist['items'])):
            track_names.append(self.playlist['items'][song]['track']['name'])
        self.track_names = track_names
        return self.track_names
    
    def get_track_artists(self):
        track_artists = []
        for song in range(len(self.playlist['items'])):
            track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
        self.track_artists = track_artists
        return self.track_artists
        
    def request_song_info(self, track_name, track_artist):
        self.track_name = track_name
        self.track_artist = track_artist
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + self.genius_key}
        search_url = base_url + '/search'
        data = {'q': track_name + ' ' + track_artist}
        response = requests.get(search_url, data=data, headers=headers)
        self.response = response
        return self.response

    def check_hits(self):
        json = self.response.json()
        remote_song_info = None
        for hit in json['response']['hits']:
            if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
                remote_song_info = hit
                break
        self.remote_song_info = remote_song_info
        return self.remote_song_info
    
    def get_url(self):
        song_url = self.remote_song_info['result']['url']
        self.song_url = song_url
        return self.song_url
    
    def scrape_lyrics(self):
        page = requests.get(self.song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics1 = html.find("div", class_="lyrics")
        lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics1:
            lyrics = lyrics1.get_text()
        elif lyrics2:
            lyrics = lyrics2.get_text()
        elif lyrics1 == lyrics2 == None:
            lyrics = None
        return lyrics

    def get_lyrics(self):
        playlist = GetLyrics.get_playlist_info(self)
        track_names = GetLyrics.get_track_names(self)
        track_artists = GetLyrics.get_track_artists(self)
        song_lyrics = []
        for i in range(len(self.track_names)):
            print("\n")
            print(f"Working on track {i}.")
            response = GetLyrics.request_song_info(self, self.track_names[i], self.track_artists[i])
            remote_song_info = GetLyrics.check_hits(self)
            if remote_song_info == None:
                lyrics = None
                print(f"Track {i} is not in the Genius database.")
            else:
                url = GetLyrics.get_url(self)
                lyrics = GetLyrics.scrape_lyrics(self)
                if lyrics == None:
                    print(f"Track {i} is not in the Genius database.")
                else:
                    print(f"Retrieved track {i} lyrics!")
            song_lyrics.append(lyrics)
        return song_lyrics

In [16]:
songs = GetLyrics(cid, secret, playlist_URI, genius_key)
song_lyrics = songs.get_lyrics()

/var/folders/0q/7qqchjfd3h7gd1zmyqrf1rxw0000gn/T/ipykernel_90225/3684460710.py:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
HTTP Error for GET to https://api.spotify.com/v1/playlists/1HX3Xw3Uo9P7UHtCUh27f0/tracks with Params: {'limit': 100, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 400 due to Only valid bearer authentication supported


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/1HX3Xw3Uo9P7UHtCUh27f0/tracks?limit=100&offset=0&additional_types=track:
 Only valid bearer authentication supported, reason: None

In [21]:
%pip install lyricsgenius
from lyricsgenius import Genius

token = "-svGbhXwBRww9Fwqb1jx2C0evM4BpO9aZWQL6vfG45oF_zPaimZP7cCWkOzVfnSL"

genius = Genius(token)
artist = genius.search_artist('Death Grips')
artist.save_lyrics()

Note: you may need to restart the kernel to use updated packages.
Searching for songs by Death Grips...

Song 1: "Beware"
Song 2: "Hacker"
Song 3: "Takyon (Death Yon)"
Song 4: "Guillotine"
Song 5: "Get Got"
Song 6: "I’ve Seen Footage"
Song 7: "No Love"
Song 8: "On GP"
Song 9: "I Want It I Need It (Death Heated)"
Song 10: "You Might Think He Loves You for Your Money but I Know What He Really Loves You for It’s Your Brand New Leopard Skin Pillbox Hat"
Song 11: "The Fever (Aye Aye)"
Song 12: "Spread Eagle Cross the Block"
Song 13: "Up My Sleeves"
Song 14: "Hustle Bones"
Song 15: "Culture Shock"
Song 16: "Lord of the Game"
Song 17: "Come Up and Get Me"
Song 18: "Steroids (Crouching Tiger Hidden Gabber)"
Song 19: "Spikes"
Song 20: "Birds"
Song 21: "Inanimate Sensation"
Song 22: "Billy Not Really"
Song 23: "Punk Weight"
Song 24: "Eh"
Song 25: "Hunger Games"
Song 26: "Hot Head"
Song 27: "Lost Boys"
Song 28: "Black Quarterback"
Song 29: "Say Hey Kid"
Song 30: "Giving Bad People Good Ideas"
Son

In [23]:
df = pd.read_csv("DeathGripsDataset.csv", usecols=['names', 'albums', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo', 'types', 'duration_ms', 'time_signature', 'Labels'])
df.head()
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   names             88 non-null     object 
 1   albums            88 non-null     object 
 2   danceability      88 non-null     float64
 3   energy            88 non-null     float64
 4   key               88 non-null     int64  
 5   loudness          88 non-null     float64
 6   mode              88 non-null     int64  
 7   speechiness       88 non-null     float64
 8   acousticness      88 non-null     float64
 9   instrumentalness  88 non-null     float64
 10  valence           88 non-null     float64
 11  tempo             88 non-null     float64
 12  types             88 non-null     object 
 13  duration_ms       88 non-null     int64  
 14  time_signature    88 non-null     int64  
 15  Labels            88 non-null     int64  
dtypes: float64(8), int64(5), object(3)
memory usag

names               0
albums              0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
valence             0
tempo               0
types               0
duration_ms         0
time_signature      0
Labels              0
dtype: int64

In [25]:
h = []
for i,x in enumerate(df['names']):
    song = artist.song(x)
    h.append(song.lyrics)

Searching for "I've Seen Footage" by Death Grips...
Done.
Searching for "Come Up And Get Me" by Death Grips...
Done.
Searching for "World Of Dogs" by Death Grips...
Done.
Searching for "Bass Rattle Stars Out The Sky" by Death Grips...
Done.
Searching for "Artificial Death In The West" by Death Grips...
Done.
Searching for "You Might Think He Loves You for Your Money but I Know What He Really Loves You for It's Your Brand New Leopard Skin Pillbox Hat" by Death Grips...
Done.
Searching for "This Is Violence Now (Don't Get Me Wrong)" by Death Grips...
Done.
Searching for "I'm Overflow" by Death Grips...
Done.
Searching for "Bootleg (Don't Need Your Help)" by Death Grips...
Done.
Searching for "Whatever I Want (Fuck Who's Watching)" by Death Grips...
Done.
Searching for "Have A Sad Cum BB" by Death Grips...
Done.
Searching for "I Break Mirrors With My Face In The United States" by Death Grips...
Done.
Searching for "Why A Bitch Gotta Lie" by Death Grips...
Done.
Searching for "Centuries Of

In [26]:
df['Lyrics'] = h
df.to_csv("DeathGripsDatasetLyrics.csv")